# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [3]:
import os
price_data = os.getenv('PRICE_DATA')
if price_data is None:
    raise ValueError("The PRICE_DATA environment variable is not set.")
print("Price data directory:", price_data)



Price data directory: ../../05_src/data/prices/


In [4]:
import dask.dataframe as dd

/var/folders/7x/my66ffmj71qfbtl1sp42nf840000gn/T/ipykernel_79621/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob
import dask.dataframe as dd
price_data = os.getenv('PRICE_DATA')
if not price_data:
    raise ValueError("The PRICE_DATA environment variable is not set.")

parquet_files = [path for path in glob(os.path.join(price_data, "**/*.parquet"), recursive=True) if os.path.isfile(path)]
if not parquet_files:
    raise ValueError("No Parquet files found.")
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
print(dd_px.head())


             Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      2000-01-03  56.330471  56.464592  48.193848  51.502148  43.532227   
A      2000-01-04  48.730328  49.266811  46.316166  47.567955  40.206844   
A      2000-01-05  47.389126  47.567955  43.141991  44.617310  37.712803   
A      2000-01-06  44.080830  44.349072  41.577251  42.918453  36.276844   
A      2000-01-07  42.247852  47.165592  42.203148  46.494991  39.299923   

         Volume       sector                       subsector  year  
ticker                                                              
A       4674353  Health Care  Life Sciences Tools & Services  2000  
A       4765083  Health Care  Life Sciences Tools & Services  2000  
A       5758642  Health Care  Life Sciences Tools & Services  2000  
A       2534434  Health Care  Life Sciences Tools & Services  2000  
A       2819626  Health Care  Life Sciences Tools & S

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


In [10]:
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532227,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206844,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712803,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276844,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299923,2819626,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [11]:
# Write your code below.

import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_lo_range = lambda x: (x['High'] -x['Low'])
))

/var/folders/7x/my66ffmj71qfbtl1sp42nf840000gn/T/ipykernel_79621/1992595785.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [12]:
dd_feat.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,positive_return,hi_lo_range
ticker,,,,,,,,,,,,,,
HUM,2013-01-02,69.739998,70.099998,68.099998,68.440002,62.685658,2206700,Health Care,Managed Health Care,2013,NaN,NaN,0,2.000000
HUM,2013-01-03,68.260002,68.279999,66.070000,66.250000,60.679787,2831900,Health Care,Managed Health Care,2013,68.440002,-0.031999,0,2.209999
HUM,2013-01-04,66.830002,67.750000,66.449997,67.470001,61.797218,1874400,Health Care,Managed Health Care,2013,66.250000,0.018415,1,1.300003
HUM,2013-01-07,67.139999,67.800003,66.650002,67.279999,61.623188,1415700,Health Care,Managed Health Care,2013,67.470001,-0.002816,0,1.150002
HUM,2013-01-08,67.250000,67.779999,65.989998,66.010002,60.459972,1989000,Health Care,Managed Health Care,2013,67.279999,-0.018876,0,1.790001


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.
import dask.dataframe as dd
dd_feat = dd_feat.repartition(npartitions=1000000)  # Adjust partition size to something that fits your dataset
pandas_df = dd_feat.compute()
dd_feat['rolling_average_return'] = dd_feat.groupby('ticker')['returns'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

print(pandas_df[['ticker', 'returns', 'rolling_average_return']].head())



## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to convert Dask to Pandas DataFrame as Dask itself is capable of calculating the rolling mean in Dask DataFrame. Dask has inbuilt method for mean. 
Yes, it would have been better to perform such a operation in dask as it is capble of handling large data increasing the efficiency.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.